### doc_entity
ESTABELECE RELAÇÕES ENTRE DOCUMENTOS E ENTIDADES NA BASE DE DADOS NO MYSQL. TRABALHAMOS COM O ACERVO **ANTONIO AZEREDO DA SILVEIRA, MINISTÉRIO DAS RELAÇÕES EXTERIORES**.  

AS ENTIDADES A SEREM TRABALHADAS SÃO:
* PAÍSES
* PESSOAS

In [1]:
import os
import sys
import numpy as np
import pandas as pd
import re
from IPython.display import clear_output
import csv
import xlrd
import xlwt
import getpass
from sshtunnel import SSHTunnelForwarder
import pymysql
import string
import pickle
import random

Especifica qual sistema operacional está sendo usado. Renato = Linux ; Marcelo = nt (Windows) e define variáveis a partir disso

In [3]:
if os.name == 'nt':
    ssh_priv_key = 'C:/Users/marcelo.ribeiro/Dropbox/A-Marcelo/Educação-Trabalho/2016-CPDOC/Remoto/marcelo_priv_rsa'
    ssh_user='marcelobribeiro'
    sql_user='marcelobribeiro'
    path_inputs = "C:/Users/marcelo.ribeiro/Dropbox/A-Marcelo/Educação-Trabalho/2016-CPDOC/Azeredo Papers/Processamento/text-learning-tools/inputs/"
    #path_inputs = "C:/Users/MARCELO/Dropbox/A-Marcelo/Educação-Trabalho/2016-CPDOC/Azeredo Papers/Processamento/text-learning-tools/inputs/"
    path_outputs = "C:/Users/marcelo.ribeiro/Dropbox/A-Marcelo/Educação-Trabalho/2016-CPDOC/Azeredo Papers/Processamento/text-learning-tools/outputs/"
else:
    ssh_priv_key = '/home/rsouza/.ssh/id_rsa'
    ssh_user='rsouza'
    sql_user='rsouza'
    path_inputs = "/home/rsouza/Documentos/text-learning-tools/inputs/"
    path_outputs = "/home/rsouza/Documentos/text-learning-tools/outputs/"

### Captura lista de nomes da base Accessus/CPDOC

Abre arquivo XLS

In [11]:
workbook = xlrd.open_workbook(path_inputs+'PESSOAS-do-Accessus.xls', on_demand = True)

#worksheet = workbook.sheet_by_index(0)
worksheet = workbook.sheet_by_name('Rel_DescritorEleito')

WARNING *** file size (1367936) not 512 + multiple of sector size (512)


In [12]:
def strip_accents(s):
   return ''.join(c for c in unicodedata.normalize('NFD', s)
                  if unicodedata.category(c) != 'Mn')

Retira termos obtidos da lista de nomes, obtida em teste preliminar, que não são pessoas.

In [13]:
not_persons_list = ['manifestações', 'manobras', 'mão de obra', 'marxismo', 'materialismo', 'medicina', 'medidas', 'meeiros', 'meio', 'mercado', 
 'mercantilismo', 'messianismo', 'metrologia', 'ministério', 'missão', 'mobilidade', 'mobilização', 'modelo', 'monarquia', 
 'monopólio', 'mortalidade', 'movimento', 'mudança', 'municípios', 'nação', 'nacionalismo', 'negros']

In [14]:
cpdoc_names_list = {}

for line in range(worksheet.nrows):
    breakpoint = 0
    var_names = []
    number_name = str(worksheet.cell(line, 0).value)
    check_number = re.search('^\d',number_name)
    if check_number is not None:
        if line == worksheet.nrows-1: break
        fullname = str(worksheet.cell(line, 1).value)
        fullname = str.lower(fullname)
        for not_person in not_persons_list:
            if not_person in fullname: breakpoint = 1
        if breakpoint == 1: continue
        fullname_strip_accent = strip_accents(fullname)
        fullname_strip_abrev = re.sub('(\w\. ){0,4}', r'', fullname)
        fullname_strip_both = strip_accents(fullname_strip_abrev)
        var_names.append(fullname)
        if fullname_strip_accent != fullname: var_names.append(fullname_strip_accent)
        if fullname_strip_abrev  != fullname: var_names.append(fullname_strip_abrev)
        if fullname_strip_both   != fullname_strip_abrev and fullname_strip_both != fullname_strip_accent: 
            var_names.append(fullname_strip_both)
        cpdoc_names_list[fullname] = var_names

In [56]:
sorted(cpdoc_names_list)[100:110]

['adelmo lodi',
 'ademar alegria',
 'ademar braga',
 'ademar de barros',
 'ademar de barros filho',
 'ademar de faria',
 'ademar de melo ',
 'ademar de queiroz',
 'ademar galvão',
 'ademar gutierrez']

### Captura lista de nomes da base History-Lab.

In [91]:
count = 0
texts = []
frus_names_list = {}

#pass_sshkey = getpass.getpass()
pass_mysql = getpass.getpass()
conn = pymysql.connect(host='history-lab.org', 
                       user='de_reader',
                       passwd=pass_mysql,
                       db='declassification_frus',
                       use_unicode=True, 
                       charset="utf8")
cur = conn.cursor()
'''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''
insere dados na base mysql
captura documentos da tabela docs, transfere para a tabela topic-doc e insere dados de tópicos
'''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''

cur.execute("SELECT id, name FROM persons")
data = cur.fetchall()

for row in data:
    if row is None: break
    var_names = []
    count += 1 # for break event
    fullname =  row[1]
    fullname = str.lower(fullname)
    fullname = re.sub('(.*), (.*)', r'\2 \1', fullname)
    fullname = re.sub('(.*)\(.*\)\s?(.*)', r'\1\2', fullname)
    fullname = re.sub('- ', '-', fullname)
    fullname_strip_accent = strip_accents(fullname)
    fullname_strip_abrev = re.sub('(\w\. ){0,4}', r'', fullname)
    fullname_strip_both = strip_accents(fullname_strip_abrev)
    var_names.append(fullname)
    if fullname_strip_accent != fullname: var_names.append(fullname_strip_accent)
    if fullname_strip_abrev  != fullname: var_names.append(fullname_strip_abrev)
    if fullname_strip_both   != fullname_strip_abrev and fullname_strip_both != fullname_strip_accent: 
        var_names.append(fullname_strip_both)
    frus_names_list[fullname] = var_names    
    #if count == 10: break #amostra de apenas x linhas

········


In [93]:
len(frus_names_list)

16284

### Cria lista de nomes duplicados e remove das outras listas

In [24]:
duplicate_names_list = {}
for name, values in cpdoc_names_list.items():
    for alt_name in values:
        if alt_name in frus_names_list:
            duplicate_names_list[name] = cpdoc_names_list[name]

Remove das outras listas

In [26]:
for name in duplicate_names_list:
    cpdoc_names_list.pop(name, None)
    frus_names_list.pop(name, None)   

### Captura textos da base CPDOC_AS do mysql.

In [19]:
count = 0
texts = ''

pass_sshkey = getpass.getpass()
pass_mysql = getpass.getpass()
with SSHTunnelForwarder(('200.20.164.147', 22),
                        ssh_private_key=ssh_priv_key,
                        ssh_private_key_password=pass_sshkey,
                        ssh_username=ssh_user,
                        remote_bind_address=('127.0.0.1', 3306)) as server:
    
    conn = pymysql.connect(host='localhost', 
                           port=server.local_bind_port, 
                           user=sql_user,
                           passwd=pass_mysql,
                           db='CPDOC_AS',
                           use_unicode=True, 
                           charset="utf8")
    cur = conn.cursor()

    cur.execute("SELECT id,body FROM CPDOC_AS.docs")
    data = cur.fetchall()
    
    for row in data:
        if row is None: break
        count += 1 # for break event
        text =  row[1]
        texts += '\r\n'+ text
        #if count == 10: break #amostra de apenas x linhas

········
········


### Faz mineração de dados: checa a existência de cada nome das listas em todo o corpus. Cria novas listas com os nomes checados.
OBS: Essa primeira checagem captura o máximo possível de nomes, pois busca tanto por nomes quanto sobrenomes. Será necessário fazer uma segunda checagem para verificar se o nome exato ocorre, para evitar falsos positivos desnecessários e também para evitar ambiguidades.

In [ ]:
cpdoc_names_check = {}
frus_names_check = {}
duplicate_names_check = {}
collections_names_check = [cpdoc_names_check, frus_names_check, duplicate_names_check]
collections_names_list = [cpdoc_names_list, frus_names_list, duplicate_names_list]
for names_list in collections_names_list:
    count = 0
    percentil = int(len(names_list)/100)
    collections_names_list_index = collections_names_list.index(names_list)
    names_check = collections_names_check[collections_names_list_index]
    for name,alt_names in names_list.items():
        
        count += 1
        if count % percentil == 0: 
                clear_output()
                print(int(count/percentil),'% done on list',collections_names_list.index(names_list))
        
        for i in alt_names:
            if i in texts:
                names_check[name] =  names_list[name]
                break
    

In [38]:
print(len(cpdoc_names_check))
print(len(frus_names_check))
print(len(duplicate_names_check))
#sorted(duplicate_names_check)[:10]

1022
922
66


Cria lista de nomes que aparecem no corpus, mas estão fora das listas do accessus e do history-lab, ma. Nota: alguns nomes não estão identificados.

In [15]:
azeredopapers_names_check = {}
new_names = ['debernardi', 'alberto nogués', 'hans-dietrich genscher', 'allara', 'akasaka', 'sauvagnargues', 'andrei gromyko']
for name in new_names: azeredopapers_names_check[name] = [name]

In [ ]:
for name in new_names: cpdoc_names_check.pop(name, None)    

### Salva as 3 listas de nomes criadas (ou as recarrega)

In [18]:
cpdoc_names_check_file = path_outputs+'/corpus_entities_persons_list_cpdoc.pkl'
frus_names_check_file = path_outputs+'/corpus_entities_persons_list_frus.pkl'
duplicate_names_check_file = path_outputs+'/corpus_entities_persons_list_duplicate.pkl'
azeredopapers_names_check_file = path_outputs+'/corpus_entities_persons_list_azeredopapers.pkl'

In [19]:
''' caso queira carregar os arquivos '''
cpdoc_names_check = pickle.load(open(cpdoc_names_check_file, 'rb'))
frus_names_check = pickle.load(open(frus_names_check_file, 'rb'))
duplicate_names_check = pickle.load(open(duplicate_names_check_file, 'rb'))
azeredopapers_names_check = pickle.load(open(azeredopapers_names_check_file, 'rb'))

In [20]:
azeredopapers_names_check

{'akasaka': ['akasaka'],
 'alberto nogués': ['alberto nogués'],
 'allara': ['allara'],
 'andrei gromyko': ['andrei gromyko'],
 'debernardi': ['debernardi'],
 'hans-dietrich genscher': ['hans-dietrich genscher'],
 'sauvagnargues': ['sauvagnargues']}

### Faz segunda checagem de nomes (duplicidades e ambiguidades), agora com a lista de nomes presentes no corpus

Retira nomes ambíguos por não serem nomes compostos (sem sobrenome em geral), preservando exceções.

In [21]:
ambiguous_list = []
exception_names = ['hirohito', 'yoshihito', 'akihito']
for fullname in frus_names_check:
    if " " not in fullname and fullname not in exception_names:
        ambiguous_list.append(fullname)
for fullname in ambiguous_list:
    frus_names_check.pop(fullname, None)
for fullname in frus_names_check:
    if " " not in fullname:
        print(fullname)

hirohito
yoshihito
akihito


Retira mais termos que não são pessoas

In [22]:
non_names = ['moçambique', 'microfilmagem', 'marrocos', 'méxico', 'camboja', 'maçonaria', 'allara', 'niterói', 'nicarágua', 
             'misticismo', 'militarismo', 'legislação', 'burundi', 'música', 'maranhão', 'nova república', 
             'mar territorial', 'jogos olímpicos', 'américa do norte'
            ]
for i in non_names:
    cpdoc_names_check.pop(i, None)    

Lista nomes que se tornam ambíguos ao retirar as abreviações.

In [23]:
for fullname in cpdoc_names_check:
    fullname_strip_abrev = re.sub('(\w\. ){0,4}', r'', fullname)
    if " " not in fullname_strip_abrev and fullname_strip_abrev != fullname:
        print(fullname)

j. e. brant
i. c. tupper
c. k. yen
e. m. feinberg
j. h. russo
j. j. seabra
j. a. wright
j. peres
f. chermont
n. brandão
d. a. doran
e. l. resende
c. h. adams
m. france
h. s. owen
y. k. pao
a. faria
d. gregory
m. bastos
a. azevedo
b. goulart
k. kato


Corrige nomes ambíguos, escritos de outra forma.

In [24]:
#nomes duplicados 
duplicate_names_check['hirohito'] = frus_names_check['hirohito']
cpdoc_names_check.pop('imperador hirohito', None) 
frus_names_check.pop('hirohito', None) 

#ocorrências incorretas
#cpdoc_names_check.pop('albert probst', None)

['hirohito']

Verifica frequência de sobrenomes (retira primeiro nome e abreviações)

In [25]:
surnames_list = []
for fullname in cpdoc_names_check:
    fullname = re.sub('(\w\. ){0,4}', r'', fullname)
    fullname = re.sub('( d[aeo] )', r' ', fullname)
    fullname = re.sub('\w+ (.*)', r'\1', fullname)
    fullname = fullname.split(' ')
    for name in fullname:
        surnames_list.append(name)
for fullname in frus_names_check:
    fullname = re.sub('(\w\. ){0,4}', r'', fullname)
    fullname = re.sub('( d[aeo] )', r' ', fullname)
    fullname = re.sub('\w+ (.*)', r'\1', fullname)
    fullname = fullname.split(' ')
    for name in fullname:
        surnames_list.append(name)

In [26]:
surnames_list[:10]

['vilhena',
 'ferreira',
 'braga',
 'ribeiro',
 'havelange',
 'sá',
 'eugênio',
 'macedo',
 'soares',
 'almeida']

### Adiciona sobrenomes muito importantes em listas de alt_names
Obs: isso pode aumentar a frequência de falsos positivos, devido à presença de parentes, mas o importante é zerar o número de falsos negativos.

In [40]:
#imp_surnames = ['callaghan', 'stroessner', 'crimmins', 'banzer', 'golbery', 'echeverría', 'carter', 'azeredo', 'nixon', 'geisel']
imp_fullnames = ['richard nixon','jimmy carter','antônio azeredo da silveira','hugo banzer suárez','alfredo stroessner','golbery do couto e silva','john hugh crimmins','james callaghan','ernesto geisel']


collections_names_check = [cpdoc_names_check, frus_names_check, duplicate_names_check, azeredopapers_names_check]
collections_names = ['cpdoc_names_check', 'frus_names_check', 'duplicate_names_check', 'azeredopapers_names_check']
for collection in collections_names_check:
    for name,alt_names in collection.items():
        if any(fullname_check == name for fullname_check in imp_fullnames):
            if name == 'antônio azeredo da silveira' or name == 'hugo banzer suárez': 
                imp_surname = re.sub('\w+ (\w+) .*', r'\1', name)
            elif name == 'golbery do couto e silva':
                imp_surname = re.sub('^(\w+) .*', r'\1', name)
            else: 
                imp_surname = re.sub('.* (\w+)$', r'\1', name)
            alt_names.append(imp_surname)
            collection[name] = alt_names

### Organizando nomes (do cpdoc) em tabela

In [29]:
all_names_check = {}
all_names_check.update(cpdoc_names_check)
all_names_check.update(frus_names_check)
all_names_check.update(duplicate_names_check)
all_names_check.update(azeredopapers_names_check)

names_series = pd.Series(all_names_check)

In [46]:
person_id = 500000 # id do History-Lab termina em 116.000. Precisamos ter uma sequência de ID distante, tal como foi acordado entre Marcelo e Rohan
names_data = pd.DataFrame(names_series, columns=['alt_names'])
names_data.index.name = 'names'
names_data['id'] = person_id
names_data['source'] = ''
names_data['has_ambiguous_surname'] = 'False'
names_data['short_fullname'] = 'False'
names_data['short_surname'] = 'False'
names_data['important_name'] = 'False'


for row in names_data.itertuples():
    
    names_data.ix[row[0], 'id'] = str(person_id)
    person_id += 1
    
    # check if fullname is too short
    for alt_name in row[1]: 
        alt_name = re.sub('^(d[aeo]s? )', '', alt_name)
        alt_name = re.sub('( d[aeo]s? )', ' ', alt_name)
        if " " not in alt_name: names_data.ix[row[0], 'short_fullname'] = 'True'
               
    # check if fullname has a too ambiguous surname, or if fullname has a too short surname
    fullname = row[0]
    clean_fullname = re.sub('\w+ (.*)', r'\1', fullname)
    clean_fullname = re.sub('(\w\. ){0,4}', '', clean_fullname)
    clean_fullname = re.sub('^(d[aeo]s? )', '', clean_fullname)
    clean_fullname = re.sub('( d[aeo]s? )', ' ', clean_fullname)
    clean_fullname = clean_fullname.split(' ')
    for name in clean_fullname:
        count = 0
        for key in cpdoc_names_check.keys():
            if name in key:
                count += 1
        if count > 1: names_data.ix[row[0], 'has_ambiguous_surname'] = 'True'         
        if len(name) < 5: names_data.ix[row[0], 'short_surname'] = 'True'
            
    # check if fullname is important
    for imp_name in imp_fullnames:
        if imp_name in row[0]:
            names_data.ix[row[0], 'important_name'] = 'True'
    
    #pick source (SHOULD HAVE DONE THAT BEFORE)
    for collection in collections_names_check:
        coll_index = collections_names_check.index(collection)
        coll_name = collections_names[coll_index]
        collection_name = re.sub('(.*)_names_check', r'\1', coll_name)
        if fullname in collection.keys():
            names_data.ix[row[0], 'source'] = collection_name

In [47]:
names_data[:10]

,alt_names,id,source,has_ambiguous_surname,short_fullname,short_surname,important_name
names,,,,,,,
a. a. fisher,"[a. a. fisher, fisher]",500000,frus,False,True,False,False
a. azevedo,"[a. azevedo, azevedo]",500001,cpdoc,True,True,False,False
a. bentinck,"[a. bentinck, bentinck]",500002,frus,False,True,False,False
a. f. susin,"[a. f. susin, susin]",500003,frus,False,True,False,False
a. faria,"[a. faria, faria]",500004,cpdoc,True,True,False,False
a. i. de lima,"[a. i. de lima, de lima]",500005,cpdoc,True,True,True,False
a. j. peixoto de castro,"[a. j. peixoto de castro, peixoto de castro]",500006,cpdoc,True,False,False,False
a. l. conrad,"[a. l. conrad, conrad]",500007,frus,False,True,False,False
a. m. malik,"[a. m. malik, malik]",500008,frus,False,True,False,False


In [48]:
for name in names_data.itertuples():
    if name[0] in imp_fullnames:
        print (name)

Pandas(Index='alfredo stroessner', alt_names=['alfredo stroessner', 'stroessner', 'stroessner', 'stroessner'], id='500092', source='duplicate', has_ambiguous_surname='False', short_fullname='True', short_surname='False', important_name='True')
Pandas(Index='antônio azeredo da silveira', alt_names=['antônio azeredo da silveira', 'antonio azeredo da silveira', 'azeredo', 'azeredo', 'azeredo'], id='500160', source='cpdoc', has_ambiguous_surname='True', short_fullname='True', short_surname='False', important_name='True')
Pandas(Index='ernesto geisel', alt_names=['ernesto geisel', 'geisel', 'geisel', 'geisel'], id='500562', source='duplicate', has_ambiguous_surname='True', short_fullname='True', short_surname='False', important_name='True')
Pandas(Index='golbery do couto e silva', alt_names=['golbery do couto e silva', 'golbery', 'golbery', 'golbery'], id='500725', source='duplicate', has_ambiguous_surname='True', short_fullname='True', short_surname='True', important_name='True')
Pandas(In

### Cria lista de nomes de arquivos filtrados 

In [17]:
doc_id_list = []

pass_sshkey = getpass.getpass()
pass_mysql = getpass.getpass()
with SSHTunnelForwarder(('200.20.164.147', 22),
                        ssh_private_key=ssh_priv_key,
                        ssh_private_key_password=pass_sshkey,
                        ssh_username=ssh_user,
                        remote_bind_address=('127.0.0.1', 3306)) as server:
    
    conn = pymysql.connect(host='localhost', 
                           port=server.local_bind_port, 
                           user=sql_user,
                           passwd=pass_mysql,
                           db='CPDOC_AS',
                           use_unicode=True, 
                           charset="utf8")
    cur = conn.cursor()

    cur.execute("SELECT * FROM CPDOC_AS.docs WHERE main_language = 'pt' AND (readability > 0.4 OR readability = -1) ")  
    data = cur.fetchall()
    
    for row in data:
        if row is None: break
        doc_id =  row[0]
        doc_id = 'AAS_mre_'+doc_id+'.txt'
        doc_id_list.append(doc_id)

········
········


### salva arquivos para segunda fase de extração

In [49]:
names_dataframe_file = path_outputs+'/names_dataframe.pkl'
#doc_id_list_file = path_outputs+'/doc_id_list.pkl'

In [50]:
# Ativar esta função apenas se precisar salvar novamente as listas de nomes checados no corpus
pickle.dump(names_data, open(names_dataframe_file, 'wb'))
#pickle.dump(doc_id_list, open(doc_id_list_file, 'wb'))

In [51]:
''' caso queira carregar os arquivos '''
names_data = pickle.load(open(names_dataframe_file, 'rb'))
#doc_id_list = pickle.load(open(doc_id_list_file, 'rb'))